# 머신러닝 실습(분류)

## 1 라이브러리 및 데이터 불러오기

데이터: 성인 인구조사 소득 예측
- id : 고유의 id 값
- age : 나이
- workclass : 고용 형태
- fnlwgt : 사람 대표성을 나타내는 가중치 (final weight의 약자)
- education : 교육 수준
- education_num : 교육 수준 수치
- marital_status: 결혼 상태
- occupation : 업종
- relationship : 가족 관계
- race : 인종
- sex : 성별
- capital_gain : 양도 소득
- capital_loss : 양도 손실
- hours_per_week : 주당 근무 시간
- native_country : 국적
- income : 수익(예측해야 하는 값)
    - "> 50K" : 1
    - "<= 50K" : 0

In [1]:
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

## 2 탐색적 데이터 분석(EDA)

In [2]:
train.head(3)

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,3331,34.0,State-gov,177331,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,Black,Male,4386,0,40.0,United-States,>50K
1,19749,58.0,Private,290661,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States,<=50K
2,1157,48.0,Private,125933,Some-college,10,Widowed,Exec-managerial,Unmarried,Black,Female,0,1669,38.0,United-States,<=50K


In [3]:
test.head(3)

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,11574,39.0,State-gov,114055,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,40.0,United-States
1,15847,38.0,Private,254114,Some-college,10,Married-spouse-absent,Prof-specialty,Own-child,Black,Female,0,0,40.0,United-States
2,17655,44.0,State-gov,55395,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,NaN,United-States


In [4]:
# 데이터 크기
train.shape, test.shape

((29304, 16), (3257, 15))

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
 15  income          29304 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 3.6+ MB


In [6]:
# 기초 통계 - 수치형
train.describe()
# age 컬럼의 최솟값이 -38 (이상치로 의심)

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29304.000000,29292.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29291.000000
mean,16264.027880,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434229
std,9384.518323,13.628811,1.055250e+05,2.570824,7477.435640,401.518928,12.324036
min,0.000000,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,8145.750000,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,16253.500000,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,24374.250000,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,32560.000000,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [7]:
# 기초 통계 - 범주형
train.describe(include='O')

,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
count,27642,29304,29304,27636,29304,29304,29304,28767,29304
unique,8,16,7,14,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,20451,9449,13466,3683,11845,25022,19578,26240,22263


In [8]:
test.describe(include = 'object')
# native.country 컬럼
# 전체 데이터 수 : 3,211
# 카테고리 종류 : 37개 --> train(41개)과 차이가 있다.
# 최빈값 : United-States
# 최빈값의 빈도 : 2,930

,workclass,education,marital.status,occupation,relationship,race,sex,native.country
count,3083,3257,3257,3082,3257,3257,3257,3211
unique,8,16,7,14,6,5,2,37
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,2245,1052,1510,457,1348,2794,2212,2930


In [9]:
# 결측치 확인
# age, workclass, occupation, hours.per.week,native.country
train.isnull().sum()  

id                   0
age                 12
workclass         1662
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1668
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week      13
native.country     537
income               0
dtype: int64

In [10]:
# age, workclass, occupation, hours.per.week,native.country
test.isnull().sum()

id                  0
age                 6
workclass         174
fnlwgt              0
education           0
education.num       0
marital.status      0
occupation        175
relationship        0
race                0
sex                 0
capital.gain        0
capital.loss        0
hours.per.week      9
native.country     46
dtype: int64

In [11]:
# target별 빈도 수(카테고리별 개수 확인)
# 1. 이진 분류인지 확인
# 2. 불균형 데이터인지 확인 7:3
train['income'].value_counts()

income
<=50K    22263
>50K      7041
Name: count, dtype: int64

## 2 데이터 전처리
전처리를 최소화하고 싶다면 수치형 데이터만 선택해 모델을 학습할 수 있다.
범주형 데이터를 함께 사용한다면 인코딩이 필요하다.

### 데이터 전처리 필수/선택
    1. 결측치 : (결측치가 있다면) 필수
    2. 이상치 : 선택 (문제에서 명시하지 않으면 생략)
    3. 인코딩 : (범주형 데이터가 있다면) 필수
    4. 스케일링 : 선택
    
   머신러닝 결과를 기준으로 성능이 좋아지는 방향으로 데이터 전처리 진행
   - 빠른 진행을 위해 삭제하는 방향으로 진행
   - 모델 성능을 1차 확인 후 다시 데이터 전처리 단계에 와서 2차 결과를 확인하는 방법 추천

### 1. 결측치 처리

#### 결측치 제거

In [12]:
# 결측치 제거
print(f'처리전 {train.shape}')
# dropna() : 전체 데이터를 대상으로 결측치가 하나라도 있는 행은 모두 제거
df1 = train.dropna()
print(f'처리후 {df1.shape}')

처리전 (29304, 16)
처리후 (27096, 16)


In [13]:
df1.isnull().sum()

id                0
age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [14]:
# 결측이 있는 특정 '행'을 제거
print(f'처리전: {train.shape}')
df2 = train.dropna(subset= ['native.country','workclass'])
print(f'처리후 : {df2.shape}')
df2.isnull().sum()

처리전: (29304, 16)
처리후 : (27127, 16)


id                 0
age               12
workclass          0
fnlwgt             0
education          0
education.num      0
marital.status     0
occupation         6
relationship       0
race               0
sex                0
capital.gain       0
capital.loss       0
hours.per.week    13
native.country     0
income             0
dtype: int64

In [15]:
# 결측치가 있는 '컬럼 전체'를 삭제 (axis = 1, 기본값은 0)
print(f'처리전: {train.shape}')
df3 = train.dropna(axis=1)
print(f'처리후 : {df3.shape}')
df3.isnull().sum()

처리전: (29304, 16)
처리후 : (29304, 11)


id                0
fnlwgt            0
education         0
education.num     0
marital.status    0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
income            0
dtype: int64

In [16]:
# 특정 컬럼만 삭제(native.country, workclass)
print(f'처리전 : {train.shape}')
df4 = train.drop(['native.country','workclass'],axis = 1)
print(f'처리후 : {df4.shape}')
df4.isnull().sum()

처리전 : (29304, 16)
처리후 : (29304, 14)


id                   0
age                 12
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1668
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week      13
income               0
dtype: int64

#### 결측치 채우기 - 범주형

In [17]:
# 결측치 채우기 - 범주형(주로 최빈값으로 대체)
m =  train['workclass'].mode()
print(m)
# mode()는 시리즈 자료형으로 반환
# 빈도수가 같은 값이 1개가 아니라 여러개가 될 수도 있다.
# 1개여도 시리즈 형태로 변환(값만 변환하기 위해서 [0] 붙인다.)
m = train['workclass'].mode()[0]
print(m)

0    Private
Name: workclass, dtype: object
Private


In [18]:
# fillna(m) : 결측치를 m으로 채우기
train['workclass'] = train['workclass'].fillna(m)

m2 = train['native.country'].mode()[0]
print(m2)
train['native.country'] = train['native.country'].fillna(m2)

train.isnull().sum()

United-States


id                   0
age                 12
workclass            0
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1668
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week      13
native.country       0
income               0
dtype: int64

In [19]:
# 임의의 표시로 결측 데이터를 'X'로 채운다
# 의미가 있는 것은 아니며, 단순히 결측을 나타낸다.
# occupation
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

id                 0
age               12
workclass          0
fnlwgt             0
education          0
education.num      0
marital.status     0
occupation         0
relationship       0
race               0
sex                0
capital.gain       0
capital.loss       0
hours.per.week    13
native.country     0
income             0
dtype: int64

In [20]:
# test도 해야 한다(train의 값과 동일하게 대체한다.)
test['workclass'] = test['workclass'].fillna(m)
test['native.country'] = test['native.country'].fillna(m2)
test['occupation'] = test['occupation'].fillna('X')

test.isnull().sum()

id                0
age               6
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    9
native.country    0
dtype: int64

#### 결측치 채우기 - 수치형

In [21]:
# 평균값으로 채우기 - mean() 함수
value = int(train['age'].mean()) # 정수로 형변환
print(value)
train['age'] = train['age'].fillna(value)
test['age'] = test['age'].fillna(value)

38


In [22]:
# 중앙값으로 채우기 - median()
v2 = int(train['hours.per.week'].median())
print(v2)
train['hours.per.week'] = train['hours.per.week'].fillna(v2)
test['hours.per.week'] = test['hours.per.week'].fillna(v2)

40


In [23]:
# 결측치가 확인
print(train.isnull().sum())
print('-'*50)
print(test.isnull().sum())

id                0
age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64
--------------------------------------------------
id                0
age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
dtype: int64


### 이상치 처리

In [24]:
train[train['age']<=0][['age']]

,age
39,-33.0
79,-38.0
26161,-25.0


In [25]:
test[test['age']<=0][['age']]

,age


In [26]:
print(f'처리전 {train.shape}')
train = train[train['age']>0] # age가 1 이상의 데이터만 추출
print(f'처리후 {train.shape}')

처리전 (29304, 16)
처리후 (29301, 16)


## 3 인코딩
    데이터를 컴퓨터가 이해하고 처리할 수 있는 형식으로 변환하는 과정
    범주형(텍스트) 데이터를 숫자로 변환하는 과정
    원-핫 인코딩, 레이블 인코딩이 있다.

train은 lebel값을 포함하고 있고, label은 수치형 데이터가 아닌 범주형 데이터이므로 함께 원-핫 인코딩되는 것을 방지하기 위해 y_train변수에 담아둔다

In [28]:
y_train = train.pop('income')
len(y_train)

29301

### 원-핫 인코딩
pd.get_dummies(train) 
인코딩이 필요한 컬럼(object)만 자동으로 인코딩이 진행된 후 결과값을 반환한다.

In [29]:
train_oh = pd.get_dummies(train)
test_oh = pd.get_dummies(test)
print(train.shape,test.shape)
print(train_oh.shape,test_oh.shape)

(29301, 15) (3257, 15)
(29301, 107) (3257, 103)


In [30]:
train_oh.head(3)

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
0,3331,34.0,177331,10,4386,0,40.0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,19749,58.0,290661,9,0,0,40.0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,1157,48.0,125933,10,0,1669,38.0,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [31]:
# train 15개 컬럼 -> 107로 변경
# test 15개 컬럼 -> 103개로 변경
# native.country컬럼에서 카테고리 차이가 있었다.
# --> 원핫 인코딩 시 컬럼수의 불일치가 일어난다.
# -----> train과 test의 컬럼이 다르면 머신러닝 입력 데이터로 사용할 수 없다.
# ---------> 레이블을 합쳐서 인코딩 작업도 한다.

#### 레이블을 합쳐서 인코딩하는 방법

In [32]:
# pd.concat()을 통해 데이터를 위아래(axis=0)로 합친다.
data = pd.concat([train,test],axis=0)
print(data.shape)

# 합친 데이터를 원-핫 인코딩 한다.
data_oh = pd.get_dummies(data)

# iloc를 활용해 데이터를 분할한다. 복사본으로 만든다.
train_oh = data_oh.iloc[:len(train)].copy()
test_oh = data_oh.iloc[len(train):].copy()

print(train_oh.shape,test_oh.shape)

(32558, 15)
(29301, 107) (3257, 107)


### 레이블 인코딩
    레이블 인코딩할 object 자료형 컬럼명을 리스트 형태로 만든다.
    1. object형만 선택하는 코드를 활용하거나
    2. cols변수에 직접 리스트 형태로 만든다(컬럼 수가 많지 않으면 이 방법 사용하면 좋다, 오타 주의하기!)

In [35]:
# 방법 1
cols1 = train.select_dtypes(include='object').columns
cols1 = list(cols1)
cols1

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [34]:
# 방법 2 
cols2 = train.columns[train.dtypes == object]
cols2 = list(cols2)
cols2

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [36]:
# 방법 3
cols3 = ['workclass','education','marital.status','occupation','relationship','race','sex','native.country']
cols3

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [37]:
# !pip install scikit-learn # 머신러닝과 데이터분석을 위한 파이썬 라이브러리

In [39]:
from sklearn.preprocessing import LabelEncoder
for col in cols1:
    le = LabelEncoder() # 레이블 인코더를 불러온다.
    # fit : 매핑 사전을 만든다. ex) 빼빼로 : 1, 고래밥:2, 콘칩:3번
    # transform : 매핑 사전에 따라 데이터를 변환한다.
    # fit_transform : 이 두 작업을 한번에 처리한다.
    train[col] = le.fit_transform(train[col]) # 매핑 사전을 만든 후 사전에 따라 데이터를 변환한다.
    test[col] = le.transform(test[col]) # train과 같은 숫자로 데이터를 변환하기 위해 fit 과정 없이 transform만 용한다.


train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,3331,34.0,6,177331,15,10,2,9,0,2,1,4386,0,40.0,38
1,19749,58.0,3,290661,11,9,2,2,0,4,1,0,0,40.0,38
2,1157,48.0,3,125933,15,10,6,3,4,2,0,0,1669,38.0,38
3,693,58.0,3,100313,15,10,2,10,0,4,1,0,1902,40.0,38
4,12522,41.0,3,195661,15,10,2,13,0,4,1,0,0,54.0,38


## 4 스케일링
수치형 데이터의 범위를 조정하는 작업
    
- 작업형2에서 스케일링은 선택사항이며, 주어진 데이터와 사용하는 모델에 따라 성능이 개선될 수도 있고 아닐 수도 있다. 작업형1에도 스케일링이 출제되고는 있으니 알아두는 편이 좋다.

무난하게 풀기에는 주로 트리기반 머신러닝 모델을 사용한다.

스케일링은 트리 기반 모델에서 큰 효과를 보기 어렵다.

선형 회귀나 로지스틱 회귀와 같은 선형 모델은 스케일링의 영향을 받는다.

따라서 스케일링은 1차 제출 후 시간적 여유가 있다면 적용해 비교하는 것을 추천한다.

In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29301 entries, 0 to 29303
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29301 non-null  int64  
 1   age             29301 non-null  float64
 2   workclass       29301 non-null  int32  
 3   fnlwgt          29301 non-null  int64  
 4   education       29301 non-null  int32  
 5   education.num   29301 non-null  int64  
 6   marital.status  29301 non-null  int32  
 7   occupation      29301 non-null  int32  
 8   relationship    29301 non-null  int32  
 9   race            29301 non-null  int32  
 10  sex             29301 non-null  int32  
 11  capital.gain    29301 non-null  int64  
 12  capital.loss    29301 non-null  int64  
 13  hours.per.week  29301 non-null  float64
 14  native.country  29301 non-null  int32  
dtypes: float64(2), int32(8), int64(5)
memory usage: 2.7 MB


In [41]:
# 스케일링을 적용할 수치형 컬럼명을 리스트로 만든다.
cols = ['age','fnlwgt','education.num','capital.gain','capital.loss', 'hours.per.week']

### 민맥스 스케일링(Min-Max Scaling)
    데이터를 0과 1 사이로 변환한다.
    
    ex) 금액이 1,000원부터 100,000원까지 있다고 가정한다면 최솟값인 
    1,000원이 0이 되고, 최댓값인 100,000원이 1이 된다.

In [42]:
# 데이터 복사
train_copy = train.copy()
test_copy = test.copy()

In [43]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print(train_copy[cols].head(2))

    age  fnlwgt  education.num  capital.gain  capital.loss  hours.per.week
0  34.0  177331             10          4386             0            40.0
1  58.0  290661              9             0             0            40.0


In [44]:
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])

train_copy[cols].head(2)

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,0.232877,0.112092,0.600000,0.04386,0.0,0.397959
1,0.561644,0.189060,0.533333,0.00000,0.0,0.397959


#### 스케일링과 레이블 인코딩의 차이점
    스케일링은 다수의 컬럼에 한번에 적용 가능하다.
    레이블 인코딩은 각 컬럼마다 적용해야 하므로 여러 컬럼에 적용하기 위해서는 반복문이 꼭 필요하다.

### 스탠더드 스케일링(Standard Scaling)
    데이터를 평균이 0이고 표준편차가 1인 분포로 변환하는 방법

In [45]:
train_copy = train.copy()
test_copy = test.copy()

In [46]:
# 원본
train_copy[cols].head(2)

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


In [47]:
# 스탠더드 스케일링 후
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])

# 스케일링 후
train_copy[cols].head(2)

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.335121,-0.117705,-0.031462,0.440247,-0.216056,-0.035121
1,1.428590,0.956277,-0.420430,-0.146298,-0.216056,-0.035121


### 로버스트 스케일링 (Robust Scaling)
    각 값의 중앙값을 빼고 1사분위수(Q1)와 3사분위수(Q3)의 차이(IQR)로 나누는 방법
    다른 스케일링에 비해 이상치의 영향을 덜 받는 특징이 있다
    
    ex) capital.gain의 전과 후를 살펴보면 대부분의 사람들이 자본 이득(capital.gain)은 얻지 못했고 0에 가깝다. 
    capital.gain의 Q1과 Q3가 0에 가깝다면 이 컬럼의 대부분의 값들이 변하지 않을 수 있다.

In [48]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
train[cols].head(2)

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.15,-0.008711,0.000000,4386.0,0.0,0.0
1,1.05,0.941438,-0.333333,0.0,0.0,0.0


## 5 검증 데이터 나누기

    검증 데이터를 나누는 이유는 학습된 모델의 성능을 평가하고 개선하기 위함
    

train_test_split(train, target, test_size=0.2, random_state=0)
- train : X 데이터, 데이터프레임 형태로 입력한다.
- target : label(y) 데이터, 시리즈 형태로 입력한다.
- test_size=0.2 : 검증용 데이터 비율, 0.2는 전체 데이터 중에서 20%를 의미한다.
- random_state=0 : 랜덤적인 요소를 고정, 만약 작성하지 않으면 실행할 때마다 다른 값으로 나눠진다. 실행할 때마다 같은 데이터로 나누기 위해 고정이 필요하다. (숫자는 정수로 자유롭게 입력)
- 반환값은 4개 

In [49]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(train,y_train,test_size = 0.2, random_state = 0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((23440, 15), (5861, 15), (23440,), (5861,))

데이터 분할 이후 shape 보는 방법
- X_train.shape, X_val.shape의 컬럼 수(열 수)가 일치해야 한다.
- y_train.shape, y_val.shape에서 컬럼(열) 부분에 1이 나타나지 않는 시리즈형태.

## 6 머신러닝 학습 및 평가

의사결정 나무는 어떤 과정을 통해 결과가 도출되었는지를 설명하기 쉽지만,
과적합되기 쉬워 성능에 한계가 있다.

    과적합 : 모의고사를 볼 때 모의고사 문제를 모두 외우면 모의고사는 잘 풀게 되지만,
    다른 응용 문제가 출제되면 좋은 성적을 받기 어려운 상태를 말한다.
    
이를 보완하기 위해 여러 개의 모델을 학습시켜 사용한다 --> 앙상블 

앙상블 기법 2가지
> 배깅(bagging) : 대표 모델로 랜덤포레스트(Random Forest)
> 
> 부스팅(boosting) : 대표 모델로 라이트지비엠(LightGBM)

### 랜덤포레스트
- 여러 개의 의사결정 나무를 기반으로 한 앙상블 학습 알고리즘
1. rf = RandomForestClassifier(random_state=0)  # 모델 선정
2. rf.fit(X_train, y_train)  # 학습 진행
3. pred = rf.predict_proba(X_val)  # 예측

예측
- predict : 예측된 각 레이블(클래스)을 반환 (ex. '<=50K', '>50K')
- predict_proba : 각 레이블에 속할 확률값을 반환

In [51]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0) # 모델 선정
rf.fit(X_train,y_train) # 학습 진행
pred = rf.predict_proba(X_val) # 각 레이블에 속할 확률값 반환

print(rf.classes_)
pred[:10]

['<=50K' '>50K']


array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.9 , 0.1 ],
       [0.63, 0.37],
       [1.  , 0.  ],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.94, 0.06],
       [0.12, 0.88],
       [0.88, 0.12]])

## 7 평가 지표
- 머신러닝 모델을 학습하고 나서 제대로 학습이 되었고, 예측을 하고 있는지 검증 데이터로 평가가 필요하다
- 분류 모델의 평가지표에는 여러 가지가 있는데, 문제에서 묻는 부분을 진행한다.

### ROC_AUC
- roc_auc 점수를 평가할 때는 확률값이 필요하다.
- predict_proba() 사용

In [52]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_val, pred[:,1]) # 두번째열(1열) -> 50K보다 큰값
print(f'roc_auc : {roc_auc}')

roc_auc : 0.9173623004487484


### Accuracy (정확도)
- predict() 사용
- 예측된 각 클래스(레이블)을 반환

In [53]:
pred = rf.predict(X_val)
pred[:10]

array(['<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '>50K', '<=50K'], dtype=object)

In [54]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_val,pred)
print(f'정확도 : {accuracy}')

정확도 : 0.8694761986009213


### F1 스코어
- 정밀도(Precision)와 재현율(Recall)의 조화 평균으로 계산되는 평가지표이다.
- 높을수록 좋다.
- 기본적으로 양성 클래스를 1로 가정하는데, 어떤 값이 양성 클래스인지 pos_label='>50K'를 사용해서 지정한다. 만약 타겟이 0, 1로 변환된 숫자였다면 생략해도 된다.

In [55]:
from sklearn.metrics import f1_score

f1 = f1_score(y_val,pred,pos_label='>50K')
print(f'f1_score : {f1}')

f1_score : 0.6926476496584973


## 8 라이트지비엠 (LightGBM)
- 그래디언드 부스팅(Gradient Boosting) 기반의 앙상블 학습 알고리즘
- 머신러닝 모델 중 정형 데이터를 다룰 때 매우 인기 있는 모델
- 사이킷런이 아닌 별도 라이브러리 이용

In [56]:
# !pip install lightgbm
import lightgbm as lgb
# verbose = -1 : 로그메세지 숨기기 위해
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train,y_train)
pred = lgbmc.predict_proba(X_val)

roc_auc = roc_auc_score(y_val,pred[:,1])
print(f'roc_auc : {roc_auc}')

pred = lgbmc.predict(X_val)
accuracy = accuracy_score(y_val,pred)
print(f'정확도 : {accuracy}')

f1 = f1_score(y_val,pred,pos_label='>50K')
print(f'f1_score : {f1}')

roc_auc : 0.9279535666686397
정확도 : 0.8771540692714553
f1_score : 0.7158642462509865


## 9 예측 및 결과 파일 생성
- 앞 단계에서는 검증 데이터를 활용해 예측하고 평가했으나, 이번 단계에서는 test 데이터를 예측한다.
- 학습된 LightGBM 모델을 활용해 test 데이터를 넣고 predict_proba를 실행하면 예측 확률값을 반환받는다.
- 문제에서 $50,000 이상의 확률값을 묻고 있다. 따라서 두번째값을 pred[:1]로 선택한다.

In [61]:
pred = lgbmc.predict_proba(test)
pred

array([[0.89924007, 0.10075993],
       [0.97622077, 0.02377923],
       [0.9853122 , 0.0146878 ],
       ...,
       [0.93528667, 0.06471333],
       [0.98996872, 0.01003128],
       [0.96723292, 0.03276708]])

In [62]:
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv('result.csv',index = False)

In [63]:
df_submit = pd.read_csv('result.csv')
df_submit.head(3)

,pred
0,0.100760
1,0.023779
2,0.014688
